In [ ]:
# 注意，这是分类问题的梯度提升树
# 回归问题的梯度提升树是一个残差学习过程，见统计学习方法

In [1]:
import time
import numpy as np

In [2]:
def load_data(file_name):
    data_arr = []
    label_arr = []
    fr = open(file_name)
    
    for line in fr.readlines():
        cur_line = line.strip().split(',')
        # 将数据二值化进行处理，大于128的转成1，小雨的转成0
        data_arr.append([int(int(num) > 128) for num in curLine[1:]])
        
        if int(cur_line[0]) == 0:
            label_arr.append(1)
        else:
            label_arr.append(-1)
    return data_arr, label_arr

In [ ]:
def calc_e_Gx(train_data, train_label, n, div, rule, D):
    #初始化分类误差率为0
    e = 0
    #将训练数据矩阵中特征为n的那一列单独剥出来做成数组。因为其他元素我们并不需要，
    #直接对庞大的训练集进行操作的话会很慢
    x = train_data[:, n]
    #同样将标签也转换成数组格式，x和y的转换只是单纯为了提高运行速度
    #测试过相对直接操作而言性能提升很大
    y = train_label
    predict = []

    #依据小于和大于的标签依据实际情况会不同，在这里直接进行设置
    if rule == 'LisOne':    L = 1; H = -1
    else:                   L = -1; H = 1

    #遍历所有样本的特征m
    for i in range(train_data.shape[0]):
        if x[i] < div:
            #如果小于划分点，则预测为L
            #如果设置小于div为1，那么L就是1，
            #如果设置小于div为-1，L就是-1
            predict.append(L)
            #如果预测错误，分类错误率要加上该分错的样本的权值（8.1式）
            if y[i] != L: e += D[i]
        elif x[i] >= div:
            #与上面思想一样
            predict.append(H)
            if y[i] != H: e += D[i]
    #返回预测结果和分类错误率e
    #预测结果其实是为了后面做准备的，在算法8.1第四步式8.4中exp内部有个Gx，要用在那个地方
    #以此来更新新的D
    return np.array(predict), e

In [ ]:
def createSingleBoostingTree(train_data, train_label, D):
    m, n = np.shape(train_data)
    single_boost_tree = {}
    # 初始化分类误差率
    single_boost_tree['e'] = 1
    #对每一个特征进行遍历，寻找用于划分的最合适的特征
    for i in range(n):
        #因为特征已经经过二值化，只能为0和1，因此分切分时分为-0.5， 0.5， 1.5三挡进行切割
        for div in [-0.5, 0.5, 1.5]:
            #在单个特征内对正反例进行划分时，有两种情况：
            #可能是小于某值的为1，大于某值得为-1，也可能小于某值得是-1，反之为1
            #因此在寻找最佳提升树的同时对于两种情况也需要遍历运行
            #LisOne：Low is one：小于某值得是1
            #HisOne：High is one：大于某值得是1
            for rule in ['LisOne', 'HisOne']:
                #按照第i个特征，以值div进行切割，进行当前设置得到的预测和分类错误率
                Gx, e = calc_e_Gx(train_data, train_label, i, div, rule, D)
                #如果分类错误率e小于当前最小的e，那么将它作为最小的分类错误率保存
                if e < single_boost_tree['e']:
                    single_boost_tree['e'] = e
                    #同时也需要存储最优划分点、划分规则、预测结果、特征索引
                    #以便进行D更新和后续预测使用
                    single_boost_tree['div'] = div
                    single_boost_tree['rule'] = rule
                    single_boost_tree['Gx'] = Gx
                    single_boost_tree['feature'] = i
    #返回单层的提升树
    return single_boost_tree

In [3]:
def createBosstingTree(train_data, train_label, tree_num = 50):
    train_data = np.array(train_data)
    train_label = np.array(train_label)
    
    # 最终预测结果列表
    finall_predict = [0] * len(train_label)
    
    m, n = np.shape(train_data)
    D = [1 / m] * m  # D为训练集数据的权重, 这里进行初始化
    tree = []
    
    # 循环建立提升树
    for i in range(tree_num):
        # 建立单个树，并在当前权重D下计算分类误差率e
        cur_tree = createSingleBoostingTree(train_data, train_label, D)
        # 根据分类误差率计算当前树的权重alpha
        alpha = 1/2 * np.log((1 - cur_tree['e']) / cur_tree['e'])
        # Gx就是当前的树
        Gx = cur_tree['Gx']
        # 更新训练数据集的权值分布
        D = np.multiply(D, np.exp(-1 * alpha * np.multiply(train_label, Gx))) / sum(D)
        cur_tree['alpha'] = alpha
        tree.append(cur_tree)
        
        finall_predict += alpha * Gx  # 前向分布，加法模型
        error = sum([1 for i in range(len(train_data)) if np.sign(finall_predict[i]) != train_label[i]])
        finall_error = error / len(train_data)
        if finall_error == 0:
            return tree
        print('iter:%d:%d, sigle error:%.4f, finall error:%.4f'%(i, tree_num, cur_tree['e'], finall_error))
    
    return tree